In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [4]:
N, n = R.shape
Mf = 2
nu = 3
sigma_f = 10000
diam_X = np.sqrt(2)
rho = np.sqrt(n)

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [5]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad: pr.llo_oracle(x, r, grad, rho=rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [6]:
points = [x0]
 
for _ in range(4):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [7]:
run_alpha_policies = ["standard","line_search","icml"]

for policy in run_alpha_policies:
    policy_results = []
    for i in range(len(points)):
        x0 = points[i]
        x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,        
                        alpha_policy=policy,                                                    
                        eps=terminate_tol, 
                        print_every=10000, 
                        debug_info=False)
        policy_results.append(
            {
        'Q_hist': Q_hist,
        'time_hist': time_hist,}
        )
    with open('./results/portfolio/' + problem_name + '_' + policy + '.pckl', 'wb') as f:
        pickle.dump(policy_results, f)
#    results[problem_name][policy] = policy_results

********* Algorithm starts *********
iter = 1, stepsize = 0.6666666666666666, criterion = 0.6662498697102228, upper_bound=0.05635009446772449, lower_bound=-5.176361103704231
iter = 10000, stepsize = 0.0001999600079984003, criterion = 0.00014207315768244438, upper_bound=-4.571413460614516, lower_bound=-4.571424828101149
iter = 20000, stepsize = 9.999000099990002e-05, criterion = 7.103360912409322e-05, upper_bound=-4.5714135287153095, lower_bound=-4.571424828101149
iter = 30000, stepsize = 6.666222251849876e-05, criterion = 4.735611501862078e-05, upper_bound=-4.571413540621646, lower_bound=-4.5714157892183644
iter = 40000, stepsize = 4.999750012499375e-05, criterion = 3.5518986616161234e-05, upper_bound=-4.571413544983082, lower_bound=-4.5714157892183644
iter = 50000, stepsize = 3.999840006399744e-05, criterion = 2.841472370678842e-05, upper_bound=-4.57141354694121, lower_bound=-4.5714157892183644
130.73730039596558
********* Algorithm starts *********
iter = 1, stepsize = 0.666666666666


KeyboardInterrupt



## Run FW with LLOO

In [9]:
#run_alpha_policies = ["standard","line_search","icml"]
run_alpha_policies = ['lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       llo_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x,
                       rho=rho,
                       sigma_f=sigma_f,
                       diam_X=diam_X,                                            
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=10, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
iter = 1, stepsize = 5.379930367470755e-05, criterion = 5.376566859566106e-05, upper_bound=0.05635009446772449, lower_bound=-5.1763611036884285
iter = 10, stepsize = 5.379950645429414e-05, criterion = 5.373984370850699e-05, upper_bound=0.05381727192997366, lower_bound=-5.1751942108230065
iter = 20, stepsize = 5.379973149373576e-05, criterion = 5.371116355085553e-05, upper_bound=0.05100511365245083, lower_bound=-5.173899000173009


KeyboardInterrupt: 

## Run SCOPT

In [8]:
scopt_results = []
for i in range(len(points)):
    x0 = points[i]
    x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
            grad_x,
            hess_mult_x,
            hess_mult_vec_x,
            Mf,
            nu,
            prox_func,
            x0,  
            sc_params,                                              
            eps=terminate_tol,                                              
            print_every=1)
        
    scopt_results.append({
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    })

KeyboardInterrupt: 